#  Webscrapping empleo público

In [51]:
# Librerias
from bs4 import BeautifulSoup
import pandas as pd

ModuleNotFoundError: No module named 'readlines'

In [3]:
# Abrir archivo html
with open("input\test.html", "r", encoding="utf-8") as file:
    soup = BeautifulSoup(file, "html.parser")

In [4]:
# Identificar los elementos que tienen la etiqueta class 'caja row primerEmpleo'
divs = soup.find_all("div", {"class": "caja row primerEmpleo"})

In [6]:
# Extraer la data
data = []
for div in divs:
    # Extract job title
    job_title = div.find("div", {"id": "bx_titulos"})
    job_title = job_title.get_text(strip=True) if job_title else None

    # Extract organization
    organization = div.find("div", {"id": "bx_resumen"}).find("strong")
    organization = organization.get_text(strip=True) if organization else None

    # Extract job description
    description = div.find("div", {"id": "bx_resumen"})
    if description:
        # Remove strong tag to isolate the description
        if description.strong:
            description.strong.decompose()
        description = description.get_text(strip=True, separator="\n")

    # Extract application deadline
    deadline = div.find("div", {"id": "bx_resumen"}).find("em")
    deadline = deadline.get_text(strip=True) if deadline else None

    # Extract link to job posting
    link = div.find("a", {"class": "btnverficha"})
    link = link['href'] if link else None

    # Append extracted data to the list
    data.append({
        "Titulo": job_title,
        "Organizacion": organization,
        "Descripcion": description,
        "Deadline": deadline,
        "Link": link
    })


In [10]:
# Transformar en un pd.df
df = pd.DataFrame(data)

In [37]:
# Visualizar algunos casos
df.head(100)

,Job Title,Organization,Description,Deadline,Link
0,"01 CARGO MÉDICO CIRUJANO 11 HORAS SEMANALES, C...",Servicio de Salud Arica y Parinacota,Plazos de Postulación 07/12/2023 - 14/12/2023,Plazos de Postulación 07/12/2023 - 14/12/2023,convFicha.aspx?i=107315&c=0&j=0&tipo=avisopiza...
1,PSICÓLOGOA UNIDAD PSICOLOGÍA CLÍNICA REMPLAZO,Centro Oncológico del Norte,"Brindar atención integral y psicológica, con e...",Plazos de Postulación 07/12/2023 - 15/12/2023,convFicha.aspx?i=107332&c=0&j=0&tipo=avisopiza...
2,ABOGADO (PRESENCIAL),Centro de Formación Técnica de la Región de At...,Le corresponderá asesorar y ejecutar el princi...,Plazos de Postulación 07/12/2023 - 15/12/2023,convFicha.aspx?i=107341&c=0&j=0&tipo=avisopiza...
3,ENFERMEROA DE CONTINUIDAD PARA UNIDAD DE EMERG...,Hospital Félix Bulnes Cerda,Optimizar la gestión administrativa de la Unid...,Plazos de Postulación 07/12/2023 - 15/12/2023,convFicha.aspx?i=107358&c=0&j=0&tipo=avisopiza...
4,APOYO DE PROTECCIÓN ESPECIALIZADA - RESIDENCIA...,Dirección Nacional MEJOR NIÑEZ,Acompañar y ejecutar las actividades socio edu...,Plazos de Postulación 07/12/2023 - 28/12/2023,convFicha.aspx?i=107393&c=0&j=0&tipo=avisopiza...
...,...,...,...,...,...
95,ANALISTA DE SEGURIDAD DE INSTALACIONES RADIACT...,Comisión Chilena de Energía Nuclear,Realizar las fiscalizaciones y las evaluacione...,Plazos de Postulación 04/12/2023 - 12/12/2023,convpostularavisoTrabajo.aspx?i=106980&c=0&j=0...
96,ANALISTA PARA EL DEPARTAMENTO DE PENSIONES (CO...,Dirección de Previsión de Carabineros de Chile,Apoyar en los procesos y procedimientos asocia...,Plazos de Postulación 04/12/2023 - 12/12/2023,convpostularavisoTrabajo.aspx?i=107042&c=0&j=0...
97,PROFESIONAL ENCARGADO DE PREVENCIÓN DE RIESGO ...,Instituto Nacional de Rehabilitación Pedro Agu...,Cargo que tiene la responsabilidad de liderar ...,Plazos de Postulación 04/12/2023 - 11/12/2023,convpostularavisoTrabajo.aspx?i=107045&c=0&j=0...
98,PSICÓLOGO(A) LABORAL PARA UNIDAD DE RECLUTAMIE...,Hospital Base San José de Osorno,Aplicar técnicas psicológicas establecidas del...,Plazos de Postulación 04/12/2023 - 18/12/2023,convpostularavisoTrabajo.aspx?i=107048&c=0&j=0...


In [38]:
# Crea el link completo e indica la fecha límite
df['Link_completo'] = 'https://www.empleospublicos.cl/pub/convocatorias/' + df['Link']
df['fecha_limite'] = df['Deadline'].str.slice(-10)

In [49]:
# Exporta la convocatoria
df.to_excel("./output/df_completo_prueba.xlsx", index=False, engine='openpyxl')

In [77]:
# Definir el nombre del archivo de texto
nombre_archivo = "./input/exclusion_organizacion.txt"

# Inicializar una lista vacía para almacenar las palabras
exclusion_organizacion = []

# Abrir el archivo y leer cada línea
with open(nombre_archivo, 'r') as archivo:
    for linea in archivo:
        # Añadir la palabra a la lista, eliminando espacios en blanco y saltos de línea
        exclusion_organizacion.append(linea.strip())

# Ahora 'palabras' es una lista que contiene todas las palabras del archivo
print(exclusion_organizacion)

['hospital', 'judicial', 'asistencial', 'salud', 'contabilidad', 'finanzas', 'primaria', 'smu']


In [78]:
# Crear un vector con todos los textos para excluir del título de la oferta laboral. 

# Definir el nombre del archivo de texto
nombre_archivo = "./input/exclusion_profesiones.txt"

# Inicializar una lista vacía para almacenar las palabras
exclusion_profesiones = []

# Abrir el archivo y leer cada línea
with open(nombre_archivo, 'r') as archivo:
    for linea in archivo:
        # Añadir la palabra a la lista, eliminando espacios en blanco y saltos de línea
        exclusion_profesiones.append(linea.strip())

['abogado', 'enfermera', 'kinesiólogo', 'psicólogo', 'juridico', 'medico', 'profesor', 'medicos']


In [79]:
# Excluir del df las organizaciones que no son de interés

# Crear una máscara booleana con 'True' para las filas donde ninguna de las palabras está presente
mascara = ~df['Organizacion'].str.contains('|'.join(exclusion_organizacion), case=False, na=False)

# Filtrar el DataFrame usando la máscara
df = df[mascara]


In [80]:
# Excluir del df las organizaciones que no son de interés

# Crear una máscara booleana con 'True' para las filas donde ninguna de las palabras está presente
mascara = ~df['Titulo'].str.contains('|'.join(exclusion_profesiones), case=False, na=False)

# Filtrar el DataFrame usando la máscara
df = df[mascara]

In [82]:
# Selecciona las columnas relevantes y exporta el df filtrado
df[["Titulo", "Organizacion", "Descripcion", "Link_completo", "fecha_limite"]].to_excel("./output/df_filtrado_prueba.xlsx", index=False, engine='openpyxl')